<a href="https://colab.research.google.com/github/gyimesbalint/DL_project/blob/main/fitting_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing datasets

In [ ]:
# https://drive.google.com/file/d/1pmNSD1nbYHEAiP065s4akRXHMWFs9Dqw/view?usp=sharing DBpedia train.csv
# https://drive.google.com/file/d/1mKededzdbJsWQnwsu-R_WSILYSvNEY7c/view?usp=sharing DBpedia test.csv
!pip install gdown
!gdown --id 1pmNSD1nbYHEAiP065s4akRXHMWFs9Dqw --output train.csv  #import train.csv from drive
!gdown --id 1mKededzdbJsWQnwsu-R_WSILYSvNEY7c --output test.csv   #import test.csv from drive

Downloading...
From: https://drive.google.com/uc?id=1pmNSD1nbYHEAiP065s4akRXHMWFs9Dqw
To: /content/train.csv
100% 174M/174M [00:01<00:00, 161MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mKededzdbJsWQnwsu-R_WSILYSvNEY7c
To: /content/test.csv
100% 21.8M/21.8M [00:00<00:00, 133MB/s]


In [ ]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('train.csv', encoding='utf8',header=None) #read csv to dataframe
test_data = pd.read_csv('test.csv', encoding='utf8',header=None)   #read csv to dataframe

train_data.where(train_data[0] < 6, inplace = True)  #select first 5 categories
train_data = train_data[train_data[0].notnull()]     #remove NaN values

test_data.where(test_data[0] < 6, inplace = True)    #select first 5 categories
test_data = test_data[test_data[0].notnull()]        #remove NaN values

In [ ]:
train_data.sample(5) #sample from train data

,0,1,2
107667,3.0,Ambrose Bebb,William Ambrose Bebb (4 July 1894 – 27 April ...
168118,5.0,Malcolm Pearson Baron Pearson of Rannoch,Malcolm Everard MacLaren Pearson Baron Pearso...
69230,2.0,Strategic studies,Strategic studies is an interdisciplinary aca...
150726,4.0,Atari Bigby,Atari David Bigby (born September 19 1981) is...
36600,1.0,Roussel Uclaf,Roussel Uclaf S.A. was the second largest Fre...


In [ ]:
test_data.sample(5) #sample from test data

,0,1,2
19736,4.0,Mana Nopnech,Mana Nopnech is a professional footballer fro...
4020,1.0,Telmex,Telmex is a Mexican telecommunications compan...
14279,3.0,Chinami Tokunaga,Chinami Tokunaga (徳永 千奈美 Tokunaga Chinami) (b...
21083,5.0,Malcolm Ross (courtier),Lieutenant-Colonel Sir Walter Hugh Malcolm Ro...
16727,4.0,Emanuele Di Zenzo,Emanuele Di Zenzo (born 26 December 1979) is ...


In [ ]:
train_label = pd.to_numeric(train_data.iloc[:,0]) #select labels (int) from train data
train_text = train_data.iloc[:,1:3] #select article text from train data

In [ ]:
test_label = pd.to_numeric(test_data.iloc[:,0]) #select labels (int) from test data
test_text = test_data.iloc[:,1:3] #select article text from train data

#Pre-processing

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #vectorizer for article text data
from nltk.corpus import stopwords #stopwords for desktop usage
import nltk
nltk.download('stopwords') #stopwords for collab notebook

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords.words('english'), analyzer='word', ngram_range=(1, 1)) #vectorizer for words, removing stopwords

In [ ]:
test_title = test_text.iloc[:,0] #select titles from test text
test_desc = test_text.iloc[:,1]  #select descriptions from test text

In [ ]:
train_title = train_text.iloc[:,0] #select titles from train text
train_desc = train_text.iloc[:,1]  #select descriptions from train text

In [ ]:
titles = pd.concat([test_title, train_title]) #concat titles for vectorazition
descs = pd.concat([test_desc, train_desc]) #concat descriptions for vectorazition

In [ ]:
titles = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(titles)) #vectorizing test_title and adding it to a dataframe
title_feature_names = np.asarray(vectorizer.get_feature_names_out()) #getting all feature names for test_title
descs = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(descs)) #vectorizing test_desc and adding it to a dataframe
desc_feature_names = np.asarray(vectorizer.get_feature_names_out()) #getting all feature names for test_desc

In [ ]:
titles = titles.loc[:, titles.sum(axis=0) > 15]

In [ ]:
titles.shape

(225000, 4174)

In [ ]:
#descs = descs.loc[:, descs.sum(axis=0) > 8]

In [ ]:
#descs.shape

In [ ]:
test_title = titles[0:len(test_title.index)]
test_title.shape

(25000, 4174)

In [ ]:
#First 5 rows and feature names obtained with the vectoritzer
print(test_title[0:5])
print(title_feature_names[0:5])

   302     306     307     308     ...  127536  127570  127803  128028
0       0       0       0       0  ...       0       0       0       0
1       0       0       0       0  ...       0       0       0       0
2       0       0       0       0  ...       0       0       0       0
3       0       0       0       0  ...       0       0       0       0
4       0       0       0       0  ...       0       0       0       0

[5 rows x 4174 columns]
['002' '05' '07' '09' '0verflow']


In [ ]:
#test_desc = descs.head(len(test_desc.index))

In [ ]:
#test_desc.shape

In [ ]:
#First 5 rows and feature names obtained with the vectorizer
#print(test_desc[0:5])

In [ ]:
train_title = titles.tail(len(titles) - len(test_title))

In [ ]:
train_title.shape

(200000, 4174)

In [ ]:
#First 5 rows and feature names obtained with the vectoritzer
print(train_title[0:5])

       302     306     307     308     ...  127536  127570  127803  128028
25000       0       0       0       0  ...       0       0       0       0
25001       0       0       0       0  ...       0       0       0       0
25002       0       0       0       0  ...       0       0       0       0
25003       0       0       0       0  ...       0       0       0       0
25004       0       0       0       0  ...       0       0       0       0

[5 rows x 4174 columns]


In [ ]:
#train_desc = descs.tail(len(descs) - len(test_desc.index))

In [ ]:
#train_desc.shape

In [ ]:
#First 5 rows and feature names obtained with the vectoritzer
#print(train_desc[0:5])

In [ ]:
from scipy.sparse import csr_matrix #for min-max scaling sparse matrices

In [ ]:
def normalize(df): #function for min-max scaling of dataframes
    result = df.copy()
    for feature_name in df.columns:
        max_value = csr_matrix(df[feature_name]).max()
        min_value = csr_matrix(df[feature_name]).min()
        result[feature_name] = (df[feature_name] - min_value) / max((max_value - min_value), 1)
    return result

In [ ]:
test_title = normalize(test_title) #min-max scale test_title

In [ ]:
print(test_title[0:5])

   302     306     307     308     ...  127536  127570  127803  128028
0     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
1     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
2     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
3     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 4174 columns]


In [ ]:
#test_desc = normalize(test_desc) #min-max scale test_desc

In [ ]:
#print(test_desc[0:5])

In [ ]:
train_title = normalize(train_title) #min-max scale train_title

In [ ]:
print(train_title[0:5])

       302     306     307     308     ...  127536  127570  127803  128028
25000     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
25001     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
25002     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
25003     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
25004     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 4174 columns]


In [ ]:
#train_desc = normalize(train_desc) #min-max scale train_desc

In [ ]:
#print(train_desc[0:5])

In [ ]:
input_shape = train_title.iloc[0].shape
input_shape = (input_shape[0], 1)
input_shape

(4174, 1)

Our **X** will be the sparse matrices and the feature names from the title and the description, and **Y** will be the labels provided in the csv file.

Validation split is obtained later in the model.fit() function from training data

In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install keras-tuner

In [ ]:
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
#from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


In [ ]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv1D(
        filters=hp.Int('conv_1_filter', min_value=16, max_value=64, step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=input_shape
    ),
    keras.layers.MaxPooling1D(pool_size=1),
    keras.layers.Conv1D(
        filters=hp.Int('conv_2_filter', min_value=16, max_value=32, step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation='relu'
    ),
    keras.layers.MaxPooling1D(pool_size=1),
    keras.layers.Flatten(),
    keras.layers.Desne(
        units=hp.INt('dense_1_units',min_value=16, max_values=64, step=16),
        activation='relu'
    ),
    keras.layers.Dropout(hp.Float('dropout_1',min_value=0.25,max_value=0.75, step=0.25)),
     keras.layers.Desne(5, activation='softmax'),
  ])


  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(32, 3, activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.Conv1D(16, 3, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4172, 16)          64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1043, 16)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1041, 16)          784       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 260, 16)          0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 4160)              0         
                                                                 
 dense (Dense)               (None, 16)                6

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
patience=10
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1)
tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1)

In [ ]:
train_title.shape

(200000, 4174)

In [ ]:


partition = {}
partition['train']={0:32000}

In [ ]:
from tensorflow.keras.utils import Sequence, to_categorical

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=4, dim=(4174), n_channels=1,
                 n_classes=5, shuffle=True):
        #Initialization
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        #Generate one batch of data
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        #list_IDs_temp = [self.list_IDs.iloc[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        #Updates indexes after each epoch
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        #Generates data containing batch_size samples
        # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i in range(len(list_IDs_temp)):
            # Store sample
            X[i] = self.list_IDs.iloc[i].to_numpy().reshape(self.dim, self.n_channels)

            # Store class
            y[i] = self.labels.iloc[i]

        return X, to_categorical(y, num_classes=self.n_classes)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = DataGenerator(train_title, train_label)

In [ ]:
network_history = model.fit(train_generator, epochs=30,steps_per_epoch=250, verbose=1, callbacks=[early_stopping, checkpointer, tb])

Epoch 1/30
250/250 [==============================] - 73s 283ms/step - loss: 0.0893 - accuracy: 0.9910
Epoch 2/30
250/250 [==============================] - 70s 281ms/step - loss: 0.0100 - accuracy: 0.9970
Epoch 3/30
250/250 [==============================] - 72s 286ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 4/30
250/250 [==============================] - 71s 284ms/step - loss: 2.3655e-04 - accuracy: 1.0000
Epoch 5/30
250/250 [==============================] - 71s 283ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 6/30
250/250 [==============================] - 71s 284ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 7/30
181/250 [====================>.........] - ETA: 20s - loss: 0.0041 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [ ]:
from keras.models import load_model
model = load_model("model.hdf5")
test_err = model.evaluate(test_title, test_label)
print("Teszt hiba:", test_err[0], "Teszt pontosság:", test_err[1])

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
# először is nyerjük ki a predikciókat (valószínüség és hozzá tartozó pontosságot)
y_pred = model.predict(test_title)
y_pred = np.argmax(y_pred,1)
y_true = np.argmax(y_test,1)

print("test accuracy: %g" %(accuracy_score(y_true, y_pred)))
print("Precision", precision_score(y_true, y_pred, average="macro"))
print("Recall", recall_score(y_true, y_pred, average="macro"))
print("f1_score", f1_score(y_true, y_pred, average="macro"))
print("\nKonfúziós mátrix: ")
conf=confusion_matrix(y_true, y_pred)
print(conf)

In [ ]:
import seaborn as sns
sns.heatmap(conf, annot=True, fmt='d', vmax=20) # a vmax paraméterrel állítjuk be, hogy milyen értéktartományban jelenítse meg az adatokat